In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/raw/hallucination_annotations_xsum_summaries.csv')

In [12]:
set(df[(df.bbcid == 34687720) & (df.system == "BERTS2S")].hallucinated_span.tolist())

{'rory mcilroy will take a one-shot lead into the final round of the wgc-hsbc champions after carding a three-under',
 'the final round of the wgc-hsbc champions'}

In [19]:
from llama2_gen import load_jsonl, dump2jsonl
data = load_jsonl("data/raw/xsumfaith_val.jsonl")

In [21]:
from template import XSUM_EXTRA_TEMPLATE, SELFINST_TEMPLATE_EXTRA
import json
for line in data:
    system = line["model_name"]
    bbcid = int(line["bbcid"])
    annotation = list(set(df[(df.bbcid == bbcid) & (df.system == system)].hallucinated_span.tolist()))
    line["extra"] = xsum_annotation=json.dumps(annotation, indent=2)
dump2jsonl(data, "data/raw/xsumfaith_val_extra.jsonl")

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

MODEL = "meta-llama/Llama-2-13b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(MODEL,
                                            torch_dtype=torch.float16,
                                            trust_remote_code=True,
                                            device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)